In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [5]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [6]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

# after you pull run `ollama run llama3.2`

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [7]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI-powered tools can create high-quality content such as articles, social media posts, product descriptions, and more, saving time and resources for businesses.
2. **Image and Video Creation**: Generative AI can produce realistic images, videos, and 3D models for applications like advertising, entertainment, and product visualization.
3. **Chatbots and Virtual Assistants**: AI-powered chatbots can provide customer support, answer frequently asked questions, and offer personalized experiences, improving user engagement and satisfaction.
4. **Predictive Analytics and Forecasting**: Generative AI algorithms can analyze large datasets to predict future trends, identify patterns, and make predictions on sales, demand, and more.
5. **Personalization**: By generating content tailored to individual preferences, businesses can create a more personalized experience for customers, in

In [9]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as text, images, videos, and music that is indistinguishable from human-created content. This technology can be used for social media management, blog posts, product descriptions, and more.
2. **Marketing Automation**: Generative AI can help automate marketing tasks such as email templates, ad copywriting, and lead generation. It can also analyze customer behavior and generate personalized content recommendations.
3. **Product Design**: Generative AI can design new products using complex algorithms that simulate human-like creativity. This can be used for product designers to brainstorm ideas or create product prototypes.
4. **Customer Service Chatbots**: Generative AI can power chatbots that respond to customer inquiries with empathetic and personalized responses, improving the overall customer experience.
5. **Predictive

In [10]:
!ollama pull deepseek-r1:1.5b

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling aabd4debf0c8... 100% ▕████████████████▏ 1.1 GB                         
pulling 369ca498f347... 100% ▕████████████████▏  387 B                         
pulling 6e4c38e1172f... 100% ▕████████████████▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕████████████████▏  148 B                         
pulling a85fe2a2e58e... 100% ▕████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 
<think>
Okay, so I'm trying to understand the basics of how large language models work. I know that they're generated by training a neural network on vast amounts of text data, but I'm not exactly sure what those underpinnings are all about. The user asked for definitions related to neural networks, attention modules, and the transformer architecture.

Starting with the neural network core concepts: from what I r

In [11]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [13]:
result = Website("https://www.reddit.com/")
print(result.title)

Reddit - Dive into anything


In [14]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n"
    user_prompt += website.text
    return user_prompt

In [15]:
def messages_for(website): 
    prompt = user_prompt_for(website)
    if not prompt:
        raise ValueError("user_prompt_for(website) returned an empty string!")
    
    return [
        {"role": "system", "content": "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [16]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [17]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


display_summary("https://reddit.com")

**Summary of the Website Reddit**

* **News and Announcements**
	+ Kendrick Lamar's streams for "Not Like Us" have soared 430% on Spotify after his Super Bowl halftime show performance.
	+ The CEO of OpenAI has rejected a $97.4 billion bid from Elon Musk, saying 'no thank you'.
	+ A plane crash at the Scottsdale airport has closed the runway and left injuries unclear.
* **Popular Communities**
	+ r/AskReddit (51.4 million members) is one of the most popular communities on Reddit.
	+ r/gtaonline (1.6 million members) and r/Twitch (2.8 million members) are also among the top communities.
* **Subreddits**
	+ The website has thousands of subreddits covering various topics, including music, news, gaming, technology, pop culture, and more.

Overall, Reddit is a vast platform with a wide range of content and communities centered around different interests.